# Server

In [3]:
import tekore as tk
from dotenv import load_dotenv

from authentication.spotify_server import SpotifyServer

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)
spotify = tk.Spotify()
host = '127.0.0.1'
port = 5000

# Multiprocessing
app = SpotifyServer(host, port, spotify, cred)
token = app.spawn_single_use_server()
spotify.token = token

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2023 18:47:18] "GET /login HTTP/1.1" 307 -


# Spotify API

In [4]:
spotify.token = cred.refresh(spotify.token)

In [5]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
for track in tracks:
    print(track.name)

Quevedo: Bzrp Music Sessions, Vol. 52
Quédate - Tiësto Remix
Dancing Queen
Another One Bites The Dust - Remastered 2011
Se Voce Nao Quer Passa a Vez
Quit Playing Games (With My Heart)
Bohemian Rhapsody - Remastered 2011
Don't Stop Me Now - Remastered 2011
Quintet for Piano, Oboe, Clarinet, Horn, and Bassoon in E Flat Major, K. 452: Allegro moderato
Quintet for Piano, Oboe, Clarinet, Horn, and Bassoon in E Flat Major, K. 452: 3. Allegretto
Quintet for Piano, Oboe, Clarinet, Horn, and Bassoon in E Flat Major, K. 452: 2. Larghetto
Quintet for Piano, Oboe, Clarinet, Horn, and Bassoon in E Flat Major, K. 452: 1. Largo -
Quattro
Thank God It's Christmas
Qa bone
Quechua
Quick Musical Doodles
Quick Changements (Sonata in C)
Under Pressure - Remastered 2011
Quarantine Blues
CUFF IT
Quickie
Kings & Queens
Thank God It's Christmas - Non-Album Single
Quiver
QUARTER LIFE CRISIS
HOTEL LOBBY (Unc & Phew)
Questions
Quietly Yours
Quand vous souriez
Quieres
Dreamers [Music from the FIFA World Cup Qatar 

In [6]:
from project.util import Partition

artist_id_limit=50
artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists = [artist for partition in artists for artist in partition]
artists

[FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [5 x str]
   href = 'https://api.spotify.com/v1/artists/716NhGYqD1jl2wI1Qkgq36'
   id = '716NhGYqD1jl2wI1Qkgq36'
   images = [3 x Image(height, url, width)]
   name = 'Bizarrap'
   popularity = 86
   type = 'artist'
   uri = 'spotify:artist:716NhGYqD1jl2wI1Qkgq36',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [2 x str]
   href = 'https://api.spotify.com/v1/artists/52iwsT98xCoGgiGntTiR7K'
   id = '52iwsT98xCoGgiGntTiR7K'
   images = [3 x Image(height, url, width)]
   name = 'Quevedo'
   popularity = 87
   type = 'artist'
   uri = 'spotify:artist:52iwsT98xCoGgiGntTiR7K',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [5 x str]
   href = 'https://api.spotify.com/v1/artists/716NhGYqD1jl2wI1Qkgq36'
   id = '716NhGYqD1jl2wI1Qkgq36'
   images = [3 x Image(height, url, wi

In [7]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials

creds = LastFmCredentials("f06f4002f217a1c17ca7864a15fe059f", "00f207939f38850e1870131a03bf406a")
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

In [8]:
import model.track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists, last_network)

WSError: Track not found

# Firebase

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app(cred)

In [ ]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))